### Example usage for calculate_metrics

In [1]:
import pickle
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils.audio_to_spectrograms import create_spectrogram_pkl
#from src.main import calculate_metrics

# Copied from src.main as I'm having problems with audio mamba imports
from src.config import CLASSES
from loguru import logger
import sed_eval

def calculate_metrics(yamnet_dict, gt_event_dict, time_resolution=1.0, t_collar=0.25):
    logger.warning("Metrics not implemented!")
    # Based on event dictionary

    # Accuracy

    # IoU

    # DCASE SED eval: https://tut-arg.github.io/sed_eval/tutorial.html#id1
    event_based_metrics = sed_eval.sound_event.EventBasedMetrics(CLASSES, t_collar=t_collar)
    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(CLASSES, time_resolution=time_resolution)
    for file, estimated_event in yamnet_dict.items():
        ref_event = gt_event_dict[file]
        event_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
        segment_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(event_based_metrics)
    print(segment_based_metrics)

In [2]:
import json

In [3]:
with open('outputs/single_stage_yamnet.json', "r") as f:
    yamnet_dict = json.load(f)

In [4]:
yamnet_dict['test_snipped_scene_0000.wav']

[{'file': 'test_snipped_scene_0000.wav',
  'event_onset': 0.96,
  'event_offset': 3.84,
  'event_label': 'siren'},
 {'file': 'test_snipped_scene_0000.wav',
  'event_onset': 3.84,
  'event_offset': 5.76,
  'event_label': 'car_horn'}]

In [5]:
# check if pkl file is created
gt_pkl_path = 'data/processed/yamnet/spectrograms_test_list.pkl'
if not(os.path.exists(gt_pkl_path)):
    create_spectrogram_pkl()

In [6]:
gt_events = pickle.load(open(gt_pkl_path, 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':ref_event['event_label']}]
                    for ref_event in gt_events}

In [24]:
with open('outputs/gt_events.json', "w") as f:
    json.dump(gt_event_dict, f, indent=4)

In [7]:
gt_event_dict['test_snipped_scene_0000.wav']

[{'file': 'test_snipped_scene_0000.wav',
  'event_onset': np.float64(1.5601864044243652),
  'event_offset': np.float64(5.414698876079694),
  'event_label': 'siren'}]

In [8]:
"""with open('outputs/crnn_results.json', "r") as f:
    crnn_dict = json.load(f)
crnn_dict = {k.replace('test_scene','test_snipped_scene'):v for k, v in crnn_dict.items()}"""

'with open(\'outputs/crnn_results.json\', "r") as f:\n    crnn_dict = json.load(f)\ncrnn_dict = {k.replace(\'test_scene\',\'test_snipped_scene\'):v for k, v in crnn_dict.items()}'

In [31]:
crnn_classes = [[i['event_label'] for i in v] for k, v in crnn_dict.items()]

In [ ]:
confusion_matrix(gt_classes, crnn_classes)

In [37]:
gt_classes = [[i['event_label'] for i in v][0] for k, v in gt_event_dict.items()]

In [21]:
with open('outputs/crnn_results_merged_1.0.json', "r") as f:
    crnn_dict = json.load(f)
calculate_metrics(crnn_dict, gt_event_dict, t_collar=0.5)

2025-12-05 22:31:17.566 | WARNING  | __main__:calculate_metrics:15 - Metrics not implemented!


Event based metrics (onset-offset)
  Evaluated length                  : 4099.81 sec
  Evaluated files                   : 499 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 500.00 ms
  Offset (length)                   : 50.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 0.19 %
    Precision                       : 0.12 %
    Recall                          : 0.40 %
  Error rate
    Error rate (ER)                 : 4.26 
    Substitution rate               : 0.01 
    Deletion rate                   : 0.98 
    Insertion rate                  : 3.26 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 0.22 %
    Precision                       : 0.15 %
    Recall                          : 0.41 %
  Error rate
    Error rate (ER)                 : 4.23 
    Deletion rate                   : 1.00 
    Insertion rate       

In [22]:
with open('outputs/crnn_results_merged_0.6.json', "r") as f:
    crnn_dict = json.load(f)
calculate_metrics(crnn_dict, gt_event_dict, t_collar=0.5)

2025-12-05 22:31:29.359 | WARNING  | __main__:calculate_metrics:15 - Metrics not implemented!


Event based metrics (onset-offset)
  Evaluated length                  : 4099.81 sec
  Evaluated files                   : 499 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 500.00 ms
  Offset (length)                   : 50.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 0.17 %
    Precision                       : 0.11 %
    Recall                          : 0.40 %
  Error rate
    Error rate (ER)                 : 4.70 
    Substitution rate               : 0.01 
    Deletion rate                   : 0.98 
    Insertion rate                  : 3.70 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 0.20 %
    Precision                       : 0.14 %
    Recall                          : 0.41 %
  Error rate
    Error rate (ER)                 : 4.66 
    Deletion rate                   : 1.00 
    Insertion rate       

In [23]:
with open('outputs/crnn_results_merged_0.3.json', "r") as f:
    crnn_dict = json.load(f)
calculate_metrics(crnn_dict, gt_event_dict, t_collar=0.5)

2025-12-05 22:31:36.382 | WARNING  | __main__:calculate_metrics:15 - Metrics not implemented!


Event based metrics (onset-offset)
  Evaluated length                  : 4099.81 sec
  Evaluated files                   : 499 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 500.00 ms
  Offset (length)                   : 50.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 0.14 %
    Precision                       : 0.09 %
    Recall                          : 0.40 %
  Error rate
    Error rate (ER)                 : 5.66 
    Substitution rate               : 0.01 
    Deletion rate                   : 0.99 
    Insertion rate                  : 4.67 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 0.18 %
    Precision                       : 0.11 %
    Recall                          : 0.41 %
  Error rate
    Error rate (ER)                 : 5.62 
    Deletion rate                   : 1.00 
    Insertion rate       

In [ ]:
import numpy as np
class_to_id = {c:i for i,c in enumerate(CLASSES)}
cm = np.zeros([5,5])
for gt, pred in zip(gt_classes, crnn_classes):
    for p in pred:
        cm[class_to_id[gt], class_to_id[p]] += 1

In [53]:
import pandas as pd

In [54]:
pd.DataFrame(cm, index = CLASSES, columns = CLASSES)

,cough,dog_bark,gun_shot,siren,car_horn
cough,135.0,170.0,75.0,72.0,63.0
dog_bark,110.0,146.0,92.0,77.0,61.0
gun_shot,100.0,128.0,83.0,93.0,72.0
siren,109.0,131.0,70.0,89.0,60.0
car_horn,81.0,114.0,74.0,77.0,53.0
